In [ ]:
pip install tensorflow

# Data analysis

### Imports

In [ ]:
# Import list

import numpy as np
import tensorflow as tf
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

### Read Data 

In [ ]:
# Read from path 

PATH = "0. Projects/3/Project-III/data/sherlock-holm.es_stories_plain-text_advs.txt"

# Read the text file
with open(PATH, 'r', encoding='utf-8') as file:
    text = file.read()

### Divide the set

In [ ]:
### Try with different separators in split

# for line in text.split('.' and '?' and '!' and ':' and ';'):
# for line in text.split'\n'):
# for line in text.split('.'):
#     print(tokenizer.texts_(to_sequences([line])[0])


### Try with all replace

# text = text.replace('\n', ' ').replace('!', '.').replace(':', '.').replace('+', ' ')
# text

In [ ]:
# Try with regular expressions

DIVIDERS_ORIGINAL = "\n"
DIVIDERS_ALL = "[,.!?:;\"]|\n\n|--| and | that | which "
DIVIDERS_MIN = "[.!]|\n\n"
DIVIDERS_BAL = "[,.!?]|\n\n|--"
divide_set = False

# Split following the dividers given
text_try = re.split(DIVIDERS_ORIGINAL, text)

# Delete all the new line comments 
text_try = [el.replace('\n', '') for el in text_try]

if divide_set:
    # Delete cover of book and extra information
    text_try = text_try[22:-24]

text_try

### Tokenization

In [ ]:
# Create Tokenizer object in python

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [ ]:
# Create input-output sequences

input_sequences = []
for line in text_try:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences

In [ ]:
# Get the max value to add padding to other entries

max_sequence_len, value = max([(len(seq), seq) for seq in input_sequences])

# max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

max_sequence_len

In [ ]:
# Get index of the longest line

# token_list = []
# for line in text_try:
#     token_list.append(tokenizer.texts_to_sequences([line])[0])

# token_list.index(value)


# Print longest line

# for el in value:
#     for word, index in tokenizer.word_index.items():
#         if index == el:
#             print(word)
#             break
len(input_sequences)

### Separate input and output

In [ ]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

X

In [ ]:
y